# Trend with Tweepy

In [22]:
import tweepy
import csv
import json
import time
# pip install ISStreamer
from ISStreamer.Streamer import Streamer
from textwrap import TextWrapper
from datetime import datetime
from elasticsearch import Elasticsearch
import sys
import numpy as np
import pandas as pd

### 1. API key

In [308]:
# api_1
c_key = "cqBrLtGgcmMOVVujAhYH74sAD"
c_secret = "Kh3TjMUXXfEP2iZmJVCavjEpxWLr5yz93abe6alNejpeiVV4WY"
a_token = "403109204-fsqizf8LQPiW5edNFrBqNXOKee4beUn0bUJeK3qx"
a_token_secret = "4BXdQDvl9eiKDaGlAPiQW8811NU80jLMVvwWHO4z9URDE"

tweepy_auth = tweepy.OAuthHandler(c_key, c_secret)
tweepy_auth.set_access_token(a_token,a_token_secret)
tweepy_api = tweepy.API(tweepy_auth, timeout=3000000000)

In [309]:
# api_2
c_key2 = "eVsSW7yWRBqle5VeiLgMP3ufC"
c_secret2 = "TmReOPxOFm754hcCRTmjBsVXRDnvecZetf4N862fUDwMnPJyOr"
a_token2 = "1333282740104400897-J7ypSnwUoKbQ5L6BXpSlK6zMNThnsh"
a_token_secret2 = "T7S24JJCoV5JeHJDVYEnHfGhzNZBY4KvWkVUfylyVfCs0"

tweepy_auth2 = tweepy.OAuthHandler(c_key2, c_secret2)
tweepy_auth2.set_access_token(a_token2,a_token_secret2)
tweepy_api2 = tweepy.API(tweepy_auth2, timeout=3000000000)

### 2. Location csv

In [310]:
location_csv = pd.read_csv('location.csv')
del location_csv['Unnamed: 5']
del location_csv['Unnamed: 6']

location_csv.head()

,code,lat,lng,country,WOEID
0,AE,24.4781,54.3686,United Arab Emirates,1940330
1,AR,-34.5997,-58.3819,Argentina,23424747
2,AT,48.2083,16.3731,Austria,23424750
3,AU,-35.2931,149.1269,Australia,1099805
4,BE,50.8467,4.3517,Belgium,23424757


### 3. ElasticSearch Setting

In [376]:
kpop_list = ['bts', '방탄', 'jimin', '슈가', '제이홉', '뷔', '정국', 'suga', 'taehyung'
             'jungkook', 'bighit', 'dynamite', 'fakelove', 'jhope', 'hobi', '호석', '호비', 
            'blackpink', 'jisoo', 'killthislove', 'lovesickgirls', 'Rosé', '블랙핑크', 
             '블핑', '제니', '리사', '로제', '지수', 'howyoulikethat', 'kpop', '카이', '엑소',
            'exo', '백현', 'baekhyun', 'suho', 'chanyeol', 'sehun', 'xiumin', '시우민', '찬열']

es = Elasticsearch('3.238.129.195:9200')

### 4. filter X

In [299]:
es = Elasticsearch('localhost:9200')
trend_dict = {}

for i in range(len(location_csv)): # csv 내의 모든 location
    
    woeid = location_csv.iloc[i]['WOEID']
    
    # 해당 location의 trend 불러오기
    trends = tweepy_api.trends_place(woeid)
    trends = trends[0]
   
    # 해당 location의 위도, 경도 불러오기
    lat = location_csv.iloc[i]['lat']
    lng = location_csv.iloc[i]['lng']

    # coordinate 형식으로 만들기
    coor = {}
    coor["coordinates"] = [lng, lat]
    coor["type"] = "Point"   
    
    trends['coordinates'] = coor
    trends['locations'] = trends['locations'][0]

    for k in range(len(trends['trends'])): # 여러 개의 trend 쪼개서 elasticsearch로 넘기기
        push_trend['trends'] = trends['trends'][k]
        push_trend['as_of'] = trends['as_of']
        push_trend['created_at'] = trends['created_at']
        push_trend['locations'] = trends['locations']
        push_trend['coordinates'] = trends['coordinates']     

        es.index(index="trend", doc_type="twitter_trend", body=push_trend)

### 5. filter O

In [325]:
trend_dict = {}

for i in range(len(location_csv)): # csv 내의 모든 location
    
    woeid = location_csv.iloc[i]['WOEID']

    # 해당 location의 trend 불러오기
    trends = tweepy_api2.trends_place(woeid)
    trends = trends[0]
   
    # 해당 location의 위도, 경도 불러오기
    lat = location_csv.iloc[i]['lat']
    lng = location_csv.iloc[i]['lng']

    # coordinate 형식으로 만들기
    coor = {}
    coor["coordinates"] = [lng, lat]
    coor["type"] = "geo_point"   
    
    trends['coordinates'] = coor
    trends['locations'] = trends['locations'][0]

    for k in range(len(trends['trends'])): # 여러 개의 trend 쪼개서 elasticsearch로 넘기기
        
        # kpop_list의 단어가 포함되었는지 확인
        num = 0
        for word in kpop_list:
            if (word in trends['trends'][k]['name'].lower()):
                num += 1
        
        # 포함되었다면 elasticsearch로 넘기기
        if (num > 0):
            push_trend['trends'] = trends['trends'][k]
            push_trend['as_of'] = trends['as_of']
            push_trend['created_at'] = trends['created_at']
            push_trend['locations'] = trends['locations']
            push_trend['coordinates'] = trends['coordinates']     

            es.index(index="trend", doc_type="twitter_trend", body=push_trend)

### 6. Mapping for geo_point

mapping 시도 !
위에는 자동 mapping으로 실행한 것

https://www.elastic.co/guide/en/elasticsearch/reference/6.1/geo-point.html

In [370]:
mappings = {
    "mappings": {
        "properties": {
            "trends": {
                "properties":{
                    "name": {"type":"text"},
                    "url":{"type" : "text"},
                    "promoted_content":{"type" : "text"},
                    "query":{"type" : "text"},
                    "tweet_volume":{"type":"long"} 
                }

             },
            "as_of": {"type":"date"},
            "created_at": {"type":"date"},
            "locations": {
                "properties":{
                    "name":{"type":"text"},
                    "woeid":{"type":"long"}
                }
             },
            "coordinates": {"type":"geo_point"}
            }
        }
    }

es.indices.create(index='trend_final', body=mappings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'trend_final'}

In [373]:
trend_dict = {}

for i in range(len(location_csv)): # csv 내의 모든 location
    
    woeid = location_csv.iloc[i]['WOEID']

    # 해당 location의 trend 불러오기
    trends = tweepy_api.trends_place(woeid)
    trends = trends[0]
   
    # 해당 location의 위도, 경도 불러오기
    lat = location_csv.iloc[i]['lat']
    lng = location_csv.iloc[i]['lng']

    # coordinate 형식으로 만들기
    coor = {}
    coor['lon'] = lng
    coor['lat'] = lat

    trends['coordinates'] = coor
    trends['locations'] = trends['locations'][0]

    for k in range(len(trends['trends'])): # 여러 개의 trend 쪼개서 elasticsearch로 넘기기
        
        # kpop_list의 단어가 포함되었는지 확인
        num = 0
        for word in kpop_list:
            if (word in trends['trends'][k]['name'].lower()):
                num += 1
        
        # 포함되었다면 elasticsearch로 넘기기
        if (num > 0):
            push_trend['trends'] = trends['trends'][k]
            push_trend['as_of'] = trends['as_of']
            push_trend['created_at'] = trends['created_at']
            push_trend['locations'] = trends['locations']
            push_trend['coordinates'] = trends['coordinates']     

            es.index(index="trend_final", body=push_trend)